In [7]:
import selenium
from pprint import pprint
file = 'image_00001'
ifile = file + '.xyz'
ofile = file + '.png'
with open(ifile, 'r') as frand:
    data = frand.readlines()

In [8]:
def wheel_element(element, deltaY = 120, offsetX = 0, offsetY = 0):
  error = element._parent.execute_script("""
    var element = arguments[0];
    var deltaY = arguments[1];
    var box = element.getBoundingClientRect();
    var clientX = box.left + (arguments[2] || box.width / 2);
    var clientY = box.top + (arguments[3] || box.height / 2);
    var target = element.ownerDocument.elementFromPoint(clientX, clientY);

    for (var e = target; e; e = e.parentElement) {
      if (e === element) {
        target.dispatchEvent(new MouseEvent('mouseover', {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY}));
        target.dispatchEvent(new MouseEvent('mousemove', {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY}));
        target.dispatchEvent(new WheelEvent('wheel',     {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY, deltaY: deltaY}));
        return;
      }
    }    
    return "Element is not interactable";
    """, element, deltaY, offsetX, offsetY)
  if error:
    raise WebDriverException(error)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import time

path_to_driver = '/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/chromedriver'
driver = webdriver.Chrome(path_to_driver)
base_url = 'http://wwwtyro.github.io/speck/'
driver.get(base_url)
driver.maximize_window()
structure = driver.find_element_by_id("menu-button-structure")
structure.click()

try:
    xyz = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "xyz-data"))).send_keys(data)
    load = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "xyz-button"))).click()
    render = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "menu-button-render"))).click()
    ao_rad = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="atom-radius"]')))
    ActionChains(driver).click_and_hold(ao_rad).move_by_offset(-10,0).release().perform()
    ao_shade = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="atom-shade"]')))
    ActionChains(driver).click_and_hold(ao_shade).move_by_offset(-60,0).release().perform()
    amb_occ = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ambient-occlusion"]')))
    ActionChains(driver).click_and_hold(amb_occ).move_by_offset(17,0).release().perform()
    bright = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="brightness"]')))
    ActionChains(driver).click_and_hold(bright).move_by_offset(45,0).release().perform()
    ao_res = Select(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ao-resolution"))))
    ao_res.select_by_value('2048')
    res = Select(WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "resolution"))))
    res.select_by_value('2048')
    time.sleep(4)
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-infobars --disable-extensions --window-size=1366,768")
    # get canvas
    render = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="render-container"]')))
    # zoom in with mouse wheel, for some reason not happening by -400
    wheel_element(render, -200)
    wheel_element(render, -200)
    time.sleep(4)
    img = render.screenshot_as_png
    with open(ofile ,"wb") as fwand:
        fwand.write(img)
    structure.click() 
    
    for i in range(10,81):
        file = 'image_000'
        ifile = file + str(i) + '.xyz'
        ofile = file + str(i) + '.png'    
        with open(ifile, 'r') as frand:
            data = frand.readlines()
        driver.find_element_by_id("xyz-data").clear()
        xyz = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "xyz-data"))).send_keys(data)
        load = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "xyz-button"))).click()
        render = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "menu-button-render"))).click()
        time.sleep(4)
        options = webdriver.ChromeOptions()
        options.add_argument("--disable-infobars --disable-extensions --window-size=1366,768")
        render = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="render-container"]')))
        wheel_element(render, -200)
        wheel_element(render, -200)
        time.sleep(4)
        img = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="render-container"]'))).screenshot_as_png
        with open(ofile ,"wb") as fwand:
            fwand.write(img)
        structure.click()  
except:
    driver.quit()